In [1]:
import os
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [2]:
import os
import numpy as np
from skimage import io, transform
import random

def load_emnist_data(base_path):
    data = []
    labels = []

    train_path = os.path.join(base_path, 'Train')
    validation_path = os.path.join(base_path, 'Validation')

    def load_data_from_folder(folder_path, label):
        images = []
        files = os.listdir(folder_path)
        random.shuffle(files)  
        num_files = len(files)
        num_to_load = num_files // 20  

        for filename in files[:num_to_load]:
            if filename.endswith(('.jpg', '.png')):
                img_path = os.path.join(folder_path, filename)
                img = io.imread(img_path, as_gray=True)  
                img = transform.resize(img, (28, 28))  
                images.append(img)
                labels.append(label)
        return images

    for label in os.listdir(train_path):
        label_path = os.path.join(train_path, label)
        if os.path.isdir(label_path):
            label_value = ord(label) 
            data += load_data_from_folder(label_path, label_value)

    for label in os.listdir(validation_path):
        label_path = os.path.join(validation_path, label)
        if os.path.isdir(label_path):
            label_value = ord(label)  
            data += load_data_from_folder(label_path, label_value)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels


In [3]:
base_path = 'C:/Users/Adam/Desktop/WTUM/dataset/'

In [4]:
data, labels = load_emnist_data(base_path)

In [6]:
datan = data / 255.0

In [7]:
X_train, X_test, y_train, y_test = train_test_split(datan, labels, test_size=0.2, random_state=42)

In [8]:
unique_labels = np.unique(labels)
num_classes = len(unique_labels)

label_to_index = {label: index for index, label in enumerate(unique_labels)}
y_train_mapped = np.array([label_to_index[label] for label in y_train])
y_test_mapped = np.array([label_to_index[label] for label in y_test])

y_train = to_categorical(y_train_mapped, num_classes)
y_test = to_categorical(y_test_mapped, num_classes)

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\Adam\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 2s 948us/step - accuracy: 0.1417 - loss: 3.2156 - val_accuracy: 0.4187 - val_loss: 2.0461
Epoch 2/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 864us/step - accuracy: 0.4762 - loss: 1.8827 - val_accuracy: 0.5595 - val_loss: 1.5752
Epoch 3/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 873us/step - accuracy: 0.5906 - loss: 1.4580 - val_accuracy: 0.6276 - val_loss: 1.3338
Epoch 4/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step - accuracy: 0.6424 - loss: 1.2559 - val_accuracy: 0.6633 - val_loss: 1.2024
Epoch 5/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 857us/step - accuracy: 0.6716 - loss: 1.1456 - val_accuracy: 0.6834 - val_loss: 1.1103
Epoch 6/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step - accuracy: 0.6964 - loss: 1.0558 - val_accuracy: 0.7134 - val_loss: 1.0297
Epoch 7/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 861us/step - accuracy: 0.7244 - loss: 0.9736 - val_accuracy: 0.7259 - val_loss: 0.9702
Epoch 8/10
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 859us/step - accuracy: 0.7391 -

In [10]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


268/268 - 0s - 485us/step - accuracy: 0.7607 - loss: 0.8414

Test accuracy: 0.7607197165489197


In [43]:
model.save('C:/Users/Adam/Desktop/WTUM/snn_model.h5')